In [8]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
# CHANGE ME!
TASK_ID = 'inner_join'
AUTHOR = 'g'

In [10]:
task = Task(
    id=TASK_ID,
    description="Select all order ids from customers that live in California",
    name="Inner join on customers and orders",
    category="Aggregation",
    plan=[
        {
        "id": "order_ids",
        "description": "select order ids",
    }, {
        "id": "California",
        "description": "customer is in California",
    }, {
        "id": "join",
        "description": "join the two tables together",
    },
    ],
    sample_input={
       "customers":[{"cid":0,"customer":"A","location":"California"},{"cid":1,"customer":"B","location":"California"},
                    {"cid":2,"customer":"C","location":"Virginia"},{"cid":3,"customer":"D","location":"New York"}],
        "orders":[{"cid":0,"oid":4,"customer":"A","orderNum":"A1"},{"cid":0,"oid":5,"customer":"A","orderNum":"A2"},
                  {"cid":1,"oid":6,"customer":"B","orderNum":"B1"},
                 {"cid":1,"oid":7,"customer":"B","orderNum":"B2"},{"cid":2,"oid":8,"customer":"C","orderNum":"C1"}],
    },
    sample_output=[4,5,6,7],
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [11]:
sql = replace(prototype,
    language='sql',
    source='''  
    SELECT orders.oid 
    FROM 
    (orders INNER JOIN customers ON orders.cid = customers.cid) 
    WHERE location="California"
''')
sql.execute(task)
sql.save()

In [13]:
datalog = replace(prototype,
    language='datalog',
    source='''  
    inner_join(oid):-
    customers(cid,customer,location),
    location="California",
    orders(cid,_,oid,_).
''')
datalog.execute(task)
datalog.save()

In [14]:
pandas = replace(prototype,
    language='python-pandas',
    source='''  
def inner_join(customers,orders):
    joined = orders.join(customers, on="cid",how="inner",lsuffix="_j")
    joined = joined.loc[joined['location'] == "California"]
    return pd.DataFrame({0: list(joined.oid)})

''')
pandas.execute(task)
pandas.save()

In [15]:
functional = replace(prototype,
    language='python-functional',
    source='''  
def inner_join(customers,orders):
    oids = []
    for order in orders:
        for customer in customers:
            if customer['cid'] == order['cid']:
                if customer['location'] == "California":
                    oids.append(order['oid'])
    return oids
''')
functional.execute(task)
functional.save()

In [16]:
imperative = replace(prototype,
    language='python-imperative',
    source='''  
def inner_join(customers,orders):
    oids = []
    cids = [customer['cid'] for customer in customers if customer['location'] == "California"]
    oids = [order['oid'] for order in orders if order['cid'] in cids]
    return oids
''').load_plan()
imperative.execute(task)
imperative.save()

In [17]:
program = imperative
program.widget(task)

Output()

CodeWidget(program='{"task": "inner_join", "language": "python-imperative", "plan": {"California": [{"line": 3…